In [7]:
import pandas as pd
import requests
import numpy as np

# URL do arquivo JSON bruto no GitHub
url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/main/TelecomX_Data.json"

# Fazendo a requisição HTTP para obter o conteúdo do arquivo
response = requests.get(url)
data = response.json()

# 1. Normalização e Carga dos Dados (ETL)
flattened_data = []
for record in data:
    customer_info = record.get('customer', {})
    phone_info = record.get('phone', {})
    internet_info = record.get('internet', {})
    account_info = record.get('account', {})
    charges_info = account_info.get('Charges', {})

    flattened_record = {
        'customerID': record.get('customerID'),
        'Churn': record.get('Churn'),
        'gender': customer_info.get('gender'),
        'SeniorCitizen': customer_info.get('SeniorCitizen'),
        'Partner': customer_info.get('Partner'),
        'Dependents': customer_info.get('Dependents'),
        'tenure': customer_info.get('tenure'),
        'PhoneService': phone_info.get('PhoneService'),
        'MultipleLines': phone_info.get('MultipleLines'),
        'InternetService': internet_info.get('InternetService'),
        'OnlineSecurity': internet_info.get('OnlineSecurity'),
        'OnlineBackup': internet_info.get('OnlineBackup'),
        'DeviceProtection': internet_info.get('DeviceProtection'),
        'TechSupport': internet_info.get('TechSupport'),
        'StreamingTV': internet_info.get('StreamingTV'),
        'StreamingMovies': internet_info.get('StreamingMovies'),
        'Contract': account_info.get('Contract'),
        'PaperlessBilling': account_info.get('PaperlessBilling'),
        'PaymentMethod': account_info.get('PaymentMethod'),
        'MonthlyCharges': charges_info.get('Monthly'),
        'TotalCharges': charges_info.get('Total')
    }
    flattened_data.append(flattened_record)

df = pd.DataFrame(flattened_data)

# 2. Transformação e Limpeza dos Dados

# Substituir espaços em branco por NaN em 'TotalCharges'
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)

# Converter 'TotalCharges' para numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Preencher valores ausentes em 'TotalCharges' e 'MonthlyCharges' com a mediana
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())
df['MonthlyCharges'] = df['MonthlyCharges'].fillna(df['MonthlyCharges'].median())

# Lidar com valores ausentes na coluna 'Churn'
df['Churn'] = df['Churn'].replace('', 'No')

# Converter 'Churn' para valores numéricos
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

# Converter 'SeniorCitizen' para categórico
df['SeniorCitizen'] = df['SeniorCitizen'].astype('category')

# Exibir informações do DataFrame
print("DataFrame após tratamento:")
df.info()

# 3. Análise Exploratória de Dados (EDA)
# Análise do Churn Rate por tipo de serviço de Internet
churn_by_internet = df.groupby('InternetService')['Churn'].mean().reset_index()
churn_by_internet['ChurnRate (%)'] = (churn_by_internet['Churn'] * 100).round(2)
print("\n--- Taxa de Evasão por Tipo de Serviço de Internet ---")
print(churn_by_internet.drop(columns='Churn'))

# Análise do Churn Rate por tipo de contrato
churn_by_contract = df.groupby('Contract')['Churn'].mean().reset_index()
churn_by_contract['ChurnRate (%)'] = (churn_by_contract['Churn'] * 100).round(2)
print("\n--- Taxa de Evasão por Tipo de Contrato ---")
print(churn_by_contract.drop(columns='Churn'))

# Análise da média de 'tenure' entre clientes que cancelaram e os que não cancelaram
avg_tenure_by_churn = df.groupby('Churn')['tenure'].mean().reset_index()
avg_tenure_by_churn['tenure'] = avg_tenure_by_churn['tenure'].round(2)
avg_tenure_by_churn['Churn'] = avg_tenure_by_churn['Churn'].map({0: 'Não Cancelou', 1: 'Cancelou'})
print("\n--- Média de Tempo de Permanência (tenure) por Churn ---")
print(avg_tenure_by_churn)


DataFrame após tratamento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   customerID        7267 non-null   object  
 1   Churn             7267 non-null   int64   
 2   gender            7267 non-null   object  
 3   SeniorCitizen     7267 non-null   category
 4   Partner           7267 non-null   object  
 5   Dependents        7267 non-null   object  
 6   tenure            7267 non-null   int64   
 7   PhoneService      7267 non-null   object  
 8   MultipleLines     7267 non-null   object  
 9   InternetService   7267 non-null   object  
 10  OnlineSecurity    7267 non-null   object  
 11  OnlineBackup      7267 non-null   object  
 12  DeviceProtection  7267 non-null   object  
 13  TechSupport       7267 non-null   object  
 14  StreamingTV       7267 non-null   object  
 15  StreamingMovies   7267 non-null   object  
 1